In [2]:
from langchain_groq import ChatGroq

In [4]:
llm = ChatGroq(
    temperature=0,
    groq_api_key = "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",
    model_name = "llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [1]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/lead-data-engineer/job/R-70053")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.






















Lead Data Engineer










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a

In [9]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ###Scraped text from website:
    {page_data}
    ###Instruction:
    the scraped text is from the career's page of a website.
    your job is to extract the job postings and return them in JSON format containing 
    following keys: 'role', 'experience', 'skills', and 'description'.
    Only return the valid JSON
    ### VALID JSON(NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data })
type(res.content)

str

In [13]:
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Lead Data Engineer',
 'experience': '10+ years of technology related experience required, 6+ years of hands-on experience in data engineering',
 'skills': ['SQL',
  'Spark',
  'Python',
  'Databricks',
  'AWS',
  'Snowflake',
  'Tableau',
  'Looker'],
 'description': 'We are looking for a passionate Lead Data Engineer, to be a part of Nike’s Geographies and Marketplace Expansion (GaME) BI team, specifically serving the stores and competitive insights business teams. You will play a critical role in developing the next generation of BI solutions helping our business teams make decisions faster and better.'}

In [12]:
type(json_res)

dict

In [14]:
import pandas as pd

In [15]:
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [18]:
import uuid
import chromadb

client = chromadb.PersistentClient("vectorstore")
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row['Techstack'],
                       metadatas = {'links': row["Links"]},
                       ids = [str(uuid.uuid4())])

In [24]:
links = collection.query(query_texts=["Experience in Python", "Expertise in React"], n_results=2).get('metadatas')
links
                

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-portfolio'},
  {'links': 'https://example.com/react-native-portfolio'}]]

In [26]:
job = json_res
job['skills']

['SQL',
 'Spark',
 'Python',
 'Databricks',
 'AWS',
 'Snowflake',
 'Tableau',
 'Looker']

In [27]:
prompt_email = PromptTemplate.from_template(
"""
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
"""
)
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Data Engineering Solutions for Nike's Geographies and Marketplace Expansion (GaME) BI Team

Dear Hiring Manager,

I came across the job description for a Lead Data Engineer at Nike's Geographies and Marketplace Expansion (GaME) BI team and was impressed by the company's vision to develop the next generation of BI solutions. As a business development executive at AtliQ, an AI & Software Consulting company, I believe our team can help you achieve this goal.

With our expertise in data engineering, we can assist in designing and implementing scalable data pipelines, leveraging technologies such as SQL, Spark, Python, Databricks, AWS, Snowflake, Tableau, and Looker. Our team has a proven track record of delivering high-quality solutions that enable businesses to make data-driven decisions.

At AtliQ, we have a strong portfolio of projects that demonstrate our capabilities in data engineering and BI solutions. Some notable examples include our work in machine learning and Py